COGS 4290 RSA

David Halpern 4/4/23

# How could we know if my mind represents concepts similarly to yours?

If I see a square and you see a square, do our internal representations look the same? [Shepard & Chipman (1970)](https://www.sciencedirect.com/science/article/abs/pii/0010028570900022) note, following arguments from Wittgenstein, that there is basically no way to know. "If there is, as we suppose, some internal event that corresponds to our perception of a square, our ability to form an association between this event and the word “square” requires only that this event have a regular relation to the external object is one of causality, not of structural isomorphism. Such an event could be the activation of some group of neurons, such, perhaps, as a “cell assembly” of the sort described by Hebb (1949). To insist, in addition, that these neurons must be spatially arranged in precisely the form of a square, themselves, does not in the least help to explain how they come to trigger the naming response “square.” On the contrary, it only attempts the absurdity of putting off until later the whole process of pattern recognition that must by definition preceed the pivotal event in question. (With about as much logic, one might as well argue that the neurons that signal that the square is green should themselves be green!)"

So what can we know about the nature of your representations and my representations? Way before the advent of modern cognitive neuroscience, Shepard & Chipman tried to tackle this problem in the following way: they gave participants a deck of 105 cards with the names of two states on them. They asked the participants to sort the deck from most similar to least similar. They then did the same thing again but with pictures of the states instead of names. Using these ranks, they used a statistical procedure called multidimensional scaling to try to map out what the state representations must have looked like in order to generate this data. This is what they got:

<center>
    <img src="figs/ChipShep70_fig1.png" width=400>
</center>

Amazingly, the results looked pretty similar whether they were ranking by image or by name! And they could give a (rough) interpretation of the similarity space:

<center>
    <img src="figs/ChipShep70_fig2.png" width=400>
</center>

Most importantly, subjects were pretty consistent across not only with themselves across the image vs. name ranking but also with the group!

<center>
    <img src="figs/ChipShep70_tab1.png" width=400>
</center>

The key insight of Shepard and Chipman was to look at the "second order isomorphism" -- it's difficult to know if my representation of Louisiana is similar to yours because we can't just ask. But if both of us think Louisiana is more similar to Florida than Colorado, that gets us a lot of the way there. Of course, with modern cognitive neuroscience methods, we can now ask a new question: does my neural activity represent things similarly to yours?

# How should we measure the similarity of neural activity?

With people, we just asked which of these are more similar. But with groups of neurons or electrodes or other sources of neural measurement (e.g. fMRI voxels), there are lots of ways to talk about similarity. For instance, consider A, B, C in figure A below:

<center>
    <img src="figs/geom_distance.png" width=400>
</center>

Imagine the points A, B and C are the responses of two electrodes to seeing three names of states. How similar are each are these? One natural way to think about similarity is Euclidean distance. This is the actual distance in the plane represented by the two electrode responses in figure A. How far apart are A and C? On Measure 1, A is C and C is 2 so they are a distance "2" away. On measure two, A is 0 and C is 2 again so the same distance. If we draw a straight line between A and C, (if you remember from high school geometry class), it would have a length $\sqrt{2^2 + 2^2} = 4$. B is the same distance from A because it is a distance of 2 along Measure 1 (but in the opposite direction as C) and 2 along Measure 2. B and C are the same on Measure 2 but 4 away on Measure 1 so a line between them would have length $\sqrt{4^2 + 0^2} = 4$. So all three points are the same distance away in terms of Euclidean distance. Therefore, we might say, from the neural activity's perspective, all three states are equally similar.

However, imagine we sampled a lot of data from the two neural measures and it looks like the black dots in figure B above. The two measures tend to be correlated. In some sense, this might imply that B should be less similar to A than C because it would be rare for the measures to take on those values. A sligthly more complicated distance measure, known as Mahalanobis distance, takes into account these statistical dependencies when computing distance. There is more detail about how it's calculated in your book.

A separate concern that often comes up with neural measures is that sometimes the numbers aren't meaningful in specific ways. For instance, if two people have slightly different skull thickness then the voltage picked up by scalp EEG will vary in magnitude. If the magnitudes of the points in the figure above doubled, their Euclidean distances would be multipled by \sqrt{2}. This is not necessarily ideal for comparing across subjects and other types of similar artifacts can make it hard to compare signals across sessions from the same subject or even within session. Therefore, it is often preferable to measure similarity using angle-based measures. These types of measures, which include cosine similarity and correlation, look at the angle between the two vectors represented by two points relative to the 0 point. The cosine similarity between a point at \[1, 2\] and \[10, 20\] is 1 because these points have the exact same angle relative to the origin. The figure below, from [Walther et al. (2016)](https://www.sciencedirect.com/science/article/pii/S1053811915011258), shows how various changes to the signals affect distance-based and angle-based distance measures.

<center>
    <img src="figs/WaltEtal16_fig1_w_caption.png" width=400>
</center>

Walther et al. given an argument for why they prefer Euclidean/Mahalanobis distances to angle-based distances when investigating neural activity that is very selective for certain kinds of stimuli (e.g. FFA for faces/PPA for places) and discuss cross-validation of distances measures to reduce bias. However, these concerns are somewhat more relevant for perception studies than memory studies for reasons that are beyond the scope of this class. Because most of the historical literature has used correlation/angle-based measures, we will use that for the examples in this class.

In the 01_compute_rsa notebook, we compute the RSA matrices for several subjects in the categorized free recall dataset. In 01_analyze_rsa notebook, we try to replicate various results in the literature.

In [33]:
import numpy as np
import xarray as xr

Generate a dataset where we observe 10 neural features across 15 states

In [56]:
# mean_features = np.random.normal(size=10)

In [65]:
A = np.diag(np.ones(15))  # a diagonal covariance matrix

In [67]:
A

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0.

Add in a couple true correlations across "states"

In [71]:
A[1, 2] = A[2, 1] = .5
A[6, 8] = A[6, 8] = .8
A

array([[1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. ],
       [0. , 1. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. ],
       [0. , 0.5, 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. ],
       [0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. ],
       [0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0.8, 0. , 0. , 0. , 0. ,
        0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. ,
        0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. ,
        0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. ,
       

In [72]:
features = np.random.multivariate_normal(size=10, mean=np.zeros(15), cov=A).T

/usr/global/Anaconda/2019-10/envs/cml37/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: covariance is not positive-semidefinite.
  """Entry point for launching an IPython kernel.


Now we want to look at two sets of features and see how similar they are

In [85]:
features[1], features[2]

(array([-0.16586799, -2.32427145,  1.66364029, -0.41732581, -0.6128518 ,
         1.35842594,  0.66071094, -0.57780713,  0.30814173, -2.12911005]),
 array([-0.5961782 , -0.37687629,  0.42682412,  0.76614478, -0.98523994,
        -0.61712705, -1.27800453,  0.50912143,  0.29913025, -0.91376358]))

Generally feature 1 is negative when feature 2 is negative and positive when feature 2 is positive so they seem pretty similar (this is because we made them similar when we generated the covariance matrix). This should be reflected when we compute their correlation. [Pearson's correlation](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) is the covariance of the two variables divided by the standard deviation of each multiplied together.

Covariance is the expectation (aka expected value aka mean) of (feature 1 - mean(feature 1)) * (feature 2 - mean(feature 2). This measure is going to be high if feature 1 is usually above its mean when feature 2 is also above its mean. Lets compute it

In [90]:
np.mean(features[1]), np.mean(features[2])

(-0.22363153359033233, -0.2765969020223576)

In [88]:
(features[1] - np.mean(features[1])) * (features[2] - np.mean(features[2]))

array([-0.01846015,  0.21065089,  1.32754666, -0.2019731 ,  0.27581823,
       -0.53873826, -0.8855873 , -0.27828226,  0.3061563 ,  1.21410742])

Now we take the average of that value (but its a sample statistic so we use n-1 in the denominator, instead of n=10)

In [116]:
feat_cov = (np.sum((features[1] - np.mean(features[1])) * 
        (features[2] - np.mean(features[2]))) / (10 - 1)) 
feat_cov 

0.15680427084582751

We can check it against `numpy`'s version

In [117]:
np.cov(features[1], features[2])

array([[1.72120761, 0.15680427],
       [0.15680427, 0.51889432]])

Now just divide that by the sample standard deviations

In [105]:
feat_cov / (np.std(features[1], ddof=1) * np.std(features[2], ddof=1))

0.16592116907603172

And check that against numpy's versiosn of correlation

In [97]:
np.corrcoef(features[1], features[2])

array([[1.        , 0.16592117],
       [0.16592117, 1.        ]])

Now we want a way to compute these correlations for all of the states and keep track of all of the state info. Like we do with EEG, we'll store it using an xarray DataArray

In [118]:
states_da = xr.DataArray(data=features, 
             dims=["states", "features"], 
             coords={'features': ['feat_' + str(i) for i in range(10)],
                     'states': ['Minn.', 'Ore.', 'W.V.', 'Colo.', 'Ala.', 'Ill.',
                               'Nev.', 'Nebr.', 'Okla.', 'Ida.', 'Fla.', 'La.', 
                                'S.C.', 'Mo.', 'Me.']})

In [119]:
states_da

<xarray.DataArray (states: 15, features: 10)>
array([[ 0.45899435, -0.93142041, -0.30951413,  2.33083068,  0.19510007,
         0.16157454,  0.51939735,  0.26091164, -1.223968  , -1.6677536 ],
       [-0.16586799, -2.32427145,  1.66364029, -0.41732581, -0.6128518 ,
         1.35842594,  0.66071094, -0.57780713,  0.30814173, -2.12911005],
       [-0.5961782 , -0.37687629,  0.42682412,  0.76614478, -0.98523994,
        -0.61712705, -1.27800453,  0.50912143,  0.29913025, -0.91376358],
       [ 0.99183672,  1.30611502, -0.55839643, -0.72332362,  0.94881791,
        -0.33534583, -1.38831806, -0.52433254, -1.46704366, -1.97510351],
       [-0.8837468 , -0.01910734, -0.47328827,  1.3832397 , -1.5202887 ,
         0.5294987 , -0.10883907, -1.2984499 ,  0.4392756 , -0.14495533],
       [-0.13460054, -0.11715018,  1.48652668,  0.38774093,  1.17604172,
         0.17061893, -1.50057836, -0.24067111, -1.00478511, -1.82334881],
       [ 0.47221058, -0.58345788,  0.77704735,  0.2656331 ,  0.82293453,
        -0.0255824 , -0.14503872,  0.67888911, -1.44250891,  1.53871906],
       [ 0.53441758, -0.77126391,  2.05299818,  1.63078701, -1.22139763,
        -0.45094464,  0.39522982,  1.46407024, -0.44006448, -1.33599084],
       [-0.81039037,  0.59026568,  0.54969872, -0.61317735, -0.17007698,
         1.18359677, -0.18480327, -0.83695829,  0.48758269, -0.14784276],
       [ 0.43029812,  1.40213575, -0.83459608, -0.05588533, -2.30626414,
         2.48761551, -0.47413934, -0.48861054,  0.07934906,  1.04142245],
       [ 0.49857383, -0.12671207,  0.22653109,  0.03252893, -0.23336879,
         0.65059478, -0.28984122,  0.39985211,  0.63128014, -0.11807812],
       [-0.52499579,  0.89071211,  0.50593092,  1.31072345, -1.1661281 ,
        -0.20288977, -0.15553454, -0.81879224,  0.75417177, -1.5506556 ],
       [-0.1013143 ,  1.74465869, -2.6271648 , -0.47309278, -0.78649209,
        -0.60664796,  0.29050107,  0.72788894,  0.19526591,  1.31226529],
       [ 0.96875185,  0.14494624,  0.48717448,  0.5809024 , -1.16477475,
        -0.19315826,  1.2500587 , -1.05844761, -1.51140712, -0.84279822],
       [ 0.70455989, -0.18994871,  0.3113842 ,  2.74595881, -1.06518872,
         0.15709184, -1.49474163, -2.05735537, -0.34646229, -0.4337716 ]])
Coordinates:
  * features  (features) <U6 'feat_0' 'feat_1' 'feat_2' ... 'feat_8' 'feat_9'
  * states    (states) <U5 'Minn.' 'Ore.' 'W.V.' 'Colo.' ... 'S.C.' 'Mo.' 'Me.'

Now we can use the xr.corr function to compute the correlation of features between states but its not so straightforward because the states have the same names in both -- just plugging it in like this leads to the wrong answer because it matches the array along its dimensions. This leads to just correlates each state with itself, which is going to  always be 1

In [120]:
xr.corr(states_da, states_da, dim='features')

<xarray.DataArray (states: 15)>
array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
Coordinates:
  * states   (states) <U5 'Minn.' 'Ore.' 'W.V.' 'Colo.' ... 'S.C.' 'Mo.' 'Me.'

What we want is each state correlated with every other one. We need to rename the states variable to get the right answer

In [108]:
list_da2 = list_da.rename({'states': 'states2'})

In [109]:
states_corr = xr.corr(list_da, list_da2, dim='features')

In [121]:
states_corr

<xarray.DataArray (states: 15, states2: 15)>
array([[ 1.        ,  0.30413896,  0.27675115,  0.17731416,  0.22951838,
         0.38638182,  0.07655976,  0.58674725, -0.4445878 , -0.24078283,
        -0.04823557,  0.30657389, -0.34848162,  0.49792883,  0.48055595],
       [ 0.30413896,  1.        ,  0.16592117, -0.19249254,  0.10872746,
         0.35647403, -0.17836605,  0.49410723,  0.31118782, -0.11781037,
         0.46921666,  0.21896903, -0.80044881,  0.26158902,  0.0771344 ],
       [ 0.27675115,  0.16592117,  1.        , -0.10427327,  0.30205423,
         0.36383651, -0.17999794,  0.6859298 , -0.09888735, -0.07733809,
         0.44183653,  0.59332726, -0.27206666, -0.14364918,  0.41117532],
       [ 0.17731416, -0.19249254, -0.10427327,  1.        , -0.39948404,
         0.58899822, -0.03886445, -0.09315732, -0.02404645, -0.07755651,
        -0.0377995 ,  0.06467481, -0.02732087,  0.16527449,  0.08371291],
       [ 0.22951838,  0.10872746,  0.30205423, -0.39948404,  1.        ,
        -0.21104457, -0.43744843,  0.09211883,  0.34253681,  0.53141754,
         0.12448478,  0.68436097,  0.03812097,  0.23445117,  0.67820877],
       [ 0.38638182,  0.35647403,  0.36383651,  0.58899822, -0.21104457,
         1.        ,  0.16259829,  0.37839286,  0.1371433 , -0.37231149,
         0.09282784,  0.23973098, -0.72892808,  0.06682976,  0.30230093],
       [ 0.07655976, -0.17836605, -0.17999794, -0.03886445, -0.43744843,
         0.16259829,  1.        ,  0.08879859, -0.41643852, -0.23280824,
...
         0.1371433 , -0.41643852, -0.32508287,  1.        ,  0.46828753,
         0.21499057,  0.29609636, -0.18599923, -0.13079881,  0.00902513],
       [-0.24078283, -0.11781037, -0.07733809, -0.07755651,  0.53141754,
        -0.37231149, -0.23280824, -0.24301668,  0.46828753,  1.        ,
         0.38588827,  0.15603861,  0.40122239,  0.12892392,  0.24826907],
       [-0.04823557,  0.46921666,  0.44183653, -0.0377995 ,  0.12448478,
         0.09282784, -0.32700403,  0.23086956,  0.21499057,  0.38588827,
         1.        ,  0.1499479 , -0.21696744, -0.21524333,  0.14117998],
       [ 0.30657389,  0.21896903,  0.59332726,  0.06467481,  0.68436097,
         0.23973098, -0.67049554,  0.42295029,  0.29609636,  0.15603861,
         0.1499479 ,  1.        , -0.17871152,  0.33292204,  0.58551818],
       [-0.34848162, -0.80044881, -0.27206666, -0.02732087,  0.03812097,
        -0.72892808, -0.17328656, -0.51492122, -0.18599923,  0.40122239,
        -0.21696744, -0.17871152,  1.        , -0.20776621, -0.30069875],
       [ 0.49792883,  0.26158902, -0.14364918,  0.16527449,  0.23445117,
         0.06682976,  0.03454064,  0.45646232, -0.13079881,  0.12892392,
        -0.21524333,  0.33292204, -0.20776621,  1.        ,  0.39722079],
       [ 0.48055595,  0.0771344 ,  0.41117532,  0.08371291,  0.67820877,
         0.30230093, -0.04206964,  0.29612902,  0.00902513,  0.24826907,
         0.14117998,  0.58551818, -0.30069875,  0.39722079,  1.        ]])
Coordinates:
  * states   (states) <U5 'Minn.' 'Ore.' 'W.V.' 'Colo.' ... 'S.C.' 'Mo.' 'Me.'
  * states2  (states2) <U5 'Minn.' 'Ore.' 'W.V.' 'Colo.' ... 'S.C.' 'Mo.' 'Me.'

In [112]:
states_corr_df = states_corr.to_dataframe('corr').reset_index()
states_corr_df

,states,states2,corr
0,Minn.,Minn.,1.000000
1,Minn.,Ore.,0.304139
2,Minn.,W.V.,0.276751
3,Minn.,Colo.,0.177314
4,Minn.,Ala.,0.229518
...,...,...,...
220,Me.,Fla.,0.141180
221,Me.,La.,0.585518
222,Me.,S.C.,-0.300699
223,Me.,Mo.,0.397221


This is the same as we calculated before!

In [113]:
states_corr_df.query('states == "Ore." and states2 == "W.V."')

,states,states2,corr
17,Ore.,W.V.,0.165921
